### Import libraries

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from minisom import MiniSom
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import DBSCAN
from wordcloud import WordCloud
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from keras import layers
# from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
# from tensorflow import keras
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split as tt
import warnings
warnings.filterwarnings('ignore')

### Reading the dataset and creating function of text pre-processing

In [2]:
df = pd.read_csv("sdmn_video_details.csv")

# Stemming / Lemmatizing
words = stopwords.words("english")
port_stem = PorterStemmer()
lem = WordNetLemmatizer()


def stemming(contents):
    contents = str(contents)
    stemmed_contents = re.sub(r'[^a-zA-Z]', ' ', contents)
    # stemmed_contents = stemmed_contents.lower()
    stemmed_contents = stemmed_contents.capitalize()
    stemmed_contents = stemmed_contents.split()
    # stemmed_contents = [port_stem.stem(word) for word in stemmed_contents if word not in words]
    # stemmed_contents = [lem.lemmatize(word) for word in stemmed_contents if word not in words]
    stemmed_contents = ' '.join(stemmed_contents)
    return stemmed_contents


df["video_title"] = df["title"].apply(stemming)
# df["video_title"] = df["title"]
# print(df['video_title'])

data = list(df['video_title'])

# Convert the textual data to a vector representation using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)
df['vectorized_data'] = list(X)

In [3]:
# Cluster the data using MiniSom
som = MiniSom(3, 3, X.shape[1], sigma=1.0, learning_rate=0.5)
som.random_weights_init(X.toarray())
som.train_random(X.toarray(), 100)

# Assign each document to a cluster based on the closest neuron
clusters = {}
for i, x in enumerate(X):
    winner = som.winner(x.toarray()[0])
    if winner not in clusters:
        clusters[winner] = [i]
    else:
        clusters[winner].append(i)

for cluster, docs in clusters.items():
    print(f"Cluster {cluster}:")
    for doc in docs:
        print(f"  {data[doc]}")

print(len(clusters))

Cluster (2, 2):
  Sidemen one word interview
  Do all the sidemen think the same
  Sidemen secret santa
  Sidemen box of lies
  Sidemen vs lucky blocks in minecraft
  Sidemen cards against humanity
  Somebody call the authorities
  The worst minutes of w s s life
  The sidemen sock off
  Sidemen lose their minds playing mario
  Sidemen finish the lyric
  Sidemen pringles mukbang
  Sidemen take an iq test
  Sidemen become superheroes
  Sidemen giant jenga painful
  Sidemen pokemon smash or pass
  Sidemen google feud
  The rematch sidemen geoguessr duel
  Sidemen geoguessr bullseye mode
  Sidemen trippy olympics
  Sidemen geoguessr duel
  Brand new sidemen trivial pursuit
  Brand new sidemen shellshock
  The ultimate sidemen gameshow
  Sidemen minigolf
  Sidemen disstrack season
  The worst swat team of all time ready or not
  Brand new sidemen horror movie
  Sidemen darkest secrets revealed
  Sidemen crossing battle royale
  The sidemen rate social media apps
  Sidemen call out penaltie

In [4]:
# Cluster the data using DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=4)
dbscan.fit(X)

clusters = {}
for i, label in enumerate(dbscan.labels_):
    if label not in clusters:
        clusters[label] = [i]
    else:
        clusters[label].append(i)

print(len(clusters))

for cluster, docs in clusters.items():
    print(f"Cluster {cluster}:")
    for doc in docs:
        print(f"  {data[doc]}")

12
Cluster -1:
  Sidemen one word interview
  Sidemen among us chaos mode
  Sidemen association challenge
  Sidemen noob vs pro rocket league
  Sidemen among us sheriff role the dumbest lobby ever
  Sidemen are you smarter than a year old
  This is the greatest sidemen video ever
  Sidemen among us jester role
  Sidemen extreme cop chase challenge on gta
  Sidemen christmas mukbang
  Sidemen reverse hide seek on ksi s mega yacht
  Sidemen gta meme olympics
  Buy christmas drillings today
  Sidemen christmas drillings edm remix official music video
  Sidemen box of lies
  Harry potter s doppelg nger
  Sidemen christmas drillings house remix official lyric video
  Sidemen vs lucky blocks in minecraft
  Sidemen grenade game on gta
  Sidemen cards against humanity
  Sidemen memory test
  Sidemen get cancelled challenge
  Sidemen tinder among us edition
  Can ksi guess the real prime
  The sidemen christmas quiz
  Sidemen blind eating challenge
  The moment that cancelled vikkstar
  Sidemen

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)

# Cluster the data using LDA
lda = LatentDirichletAllocation(n_components=5)
lda.fit(X)

# Assign each document to a cluster based on the topic with the highest probability
clusters = {}
for i, x in enumerate(X):
    topic = lda.transform(x)[0].argmax()
    if topic not in clusters:
        clusters[topic] = [i]
    else:
        clusters[topic].append(i)

print(len(clusters))

for cluster, docs in clusters.items():
    print(f"Cluster {cluster}:")
    for doc in docs:
        print(f"  {data[doc]}")

5
Cluster 2:
  Sidemen one word interview
  Sidemen among us chaos mode
  Sidemen among us sheriff role the dumbest lobby ever
  Do all the sidemen think the same
  Sidemen among us jester role
  Sidemen extreme cop chase challenge on gta
  Sidemen gta meme olympics
  Sidemen vs lucky blocks in minecraft
  Sidemen grenade game on gta
  Sidemen cards against humanity
  Sidemen complete the longest race on gta
  Sidemen draw the logo challenge
  Sidemen real or fake challenge
  Somebody call the authorities
  Sidemen bully harry on gta
  Sidemen go to the world cup
  Sidemen superhero world cup
  Sidemen hilarious proximity chat on warzone
  Sidemen guess the fake logo challenge
  Sidemen vs will smith
  Sidemen human bowling challenge
  Sidemen halloween special on gta
  Sidemen run for their life on gta
  Sidemen among us but the lobby has zero iq
  Sidemen controversial tweets
  Sidemen play the gta leaks
  Returning to the sidemen house
  Sidemen forfeit golf shot it or slap it
  Har

In [6]:
# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

inputs = tokenizer(data, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
embeddings = outputs[0][:, 0, :].numpy()

kmeans = KMeans(n_clusters=20)
kmeans.fit(embeddings)

clusters = {}
for i, label in enumerate(kmeans.labels_):
    if label not in clusters:
        clusters[label] = [i]
    else:
        clusters[label].append(i)

print(len(clusters))
# Print the clusters
# for cluster, docs in clusters.items():
#     print(f"Cluster {cluster}:")
#     for doc in docs:
#         print(f"  {data[doc]}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


20


In [7]:
j = 0
for i in range(len(clusters)):
    print(f"Cluster {i}:")
    for doc in clusters[i]:
        if doc>j:
            j = doc
        print(f" {data[doc]}")
# print(j)
# print(clusters.keys())

Cluster 0:
 Sidemen go to the world cup
 Sidemen offensive try not to laugh
 Sidemen forfeit golf shot it or slap it
 Sidemen call out penalties
 Sidemen drunk bingo
 Sidemen football darts
 Sidemen find out their football stats
 Sidemen rap or crap
 Kicking jj out of the sidemen sidemen gaming
 Sidemen moo off
 Phil bullying the sidemen for minutes straight
 Sidemen try american candy
 Sidemen give advice
 Cooking with the sidemen
 Sidemen googling ourselves
 Sidemen playing with pro footballers
 Sidemen being awkward in public
 Sidemen moo off
 Sidemen most likely to challenge
 Sidemen sumo fight behind the scenes
 Sidemen mum jokes
 Sidemen make weird noises
 Sidemen throw a dart and go where it lands
 Sidemen mukbang on a bus
 Sidemen who wants to be a millionaire
 Sidemen buying one colour for hours
 Sidemens got talent
 Sidemen tinder in real life
 Sidemen blind dating
 Sidemen split or steal
 Sidemen drinking mukbang
 Sidemen blind dating
 Sidemen paintball hide and seek
 Sideme

In [8]:
# Recommendation

In [9]:
combined_features = pd.concat([df["title"], df["description"]])
combined_features = combined_features.apply(stemming)

# Compute similarity matrix
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(combined_features)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Define function to recommend similar videos
def recommend_similar_videos(video_id, cosine_sim):
    video_index = df[df["video_id"] == video_id].index[0]
    sim_scores = list(enumerate(cosine_sim[video_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    video_indices = [i[0] for i in sim_scores]
    return df.iloc[video_indices]

# Test recommendation function
f = df[df.video_id=='AtXe3DAxPZg'].title
# print(f)
recommend_similar_videos("AtXe3DAxPZg", cosine_sim)

,channel,video_id,title,description,categoryId,publishedAt,duration,tags,likes,views,comments,video_title,vectorized_data
889,Sidemen,6ZCtuKvBYcw,SIDEMEN ARE YOU SMARTER THAN A 10 YEAR OLD,👉🏻: Subscribe to our Reacts Channel: https://w...,24,2021-04-18T17:00:23Z,PT1H17M3S,"['sidemen', 'sidemen sunday', '#sidemensunday'...",532280.0,14997485.0,24053,Sidemen are you smarter than a year old,"(0, 632)\t0.3766466862199205\n (0, 1086)\t0..."
786,MoreSidemen,FO5rw8fYUtY,BEST OLD SIDEMEN MOMENTS!,BEST MOMENTS FROM OLD SIDEMEN VIDEOS - PLEASE ...,22,2018-02-21T20:45:35Z,PT13M23S,"['sidemen', 'more sidemen', 'sidemen moments',...",186710.0,7091627.0,3264,Best old sidemen moments,"(0, 595)\t0.5515053222300713\n (0, 69)\t0.4..."
692,MoreSidemen,Dr6qecvZeew,SIDEMEN REACT TO OLD VIDEOS 3!,SIDEMEN REACT TO OLD VIDEOS 3 - PLEASE LEAVE A...,22,2018-12-14T21:00:01Z,PT10M29S,NaN,24708.0,826885.0,455,Sidemen react to old videos,"(0, 749)\t0.5157314858774761\n (0, 1017)\t0..."
718,MoreSidemen,hh2-4FPtOxk,SIDEMEN REACT TO OLD VIDEOS 2!,SIDEMEN REACT TO OLD VIDEOS 2 - PLEASE LEAVE A...,22,2018-10-08T19:00:03Z,PT13M47S,NaN,32633.0,1128161.0,352,Sidemen react to old videos,"(0, 749)\t0.5157314858774761\n (0, 1017)\t0..."
1005,Sidemen,2ubwlOsUXGs,SIDEMEN REACT TO OLD VIDEOS 2,The Sidemen react to a bunch of their old vide...,24,2019-01-27T18:01:07Z,PT14M29S,"['sidemen', 'sidemen sunday', 'sidemen react t...",210875.0,8671641.0,4357,Sidemen react to old videos,"(0, 749)\t0.5157314858774761\n (0, 1017)\t0..."
1063,Sidemen,e_Ql2ThXzAM,SIDEMEN REACT TO OLD VIDEOS,SIDEMEN REACTING TO OLD SIDEMEN VIDEOS! #Sidem...,24,2018-02-04T18:00:19Z,PT15M43S,"['sidemen', 'sidemen sunday', 'sidemen sundays...",348345.0,19078502.0,4423,Sidemen react to old videos,"(0, 749)\t0.5157314858774761\n (0, 1017)\t0..."
744,MoreSidemen,N3MpGyVrwnE,SIDEMEN REACTING TO OLD VIDEOS!,SIDEMEN REACTING TO OLD VIDEOS - PLEASE LEAVE ...,22,2018-08-10T19:00:02Z,PT14M16S,NaN,52639.0,1631526.0,695,Sidemen reacting to old videos,"(0, 750)\t0.5831085995755863\n (0, 1017)\t0..."
197,MoreSidemen,S_IOdiORI2o,SIDEMEN 8 YEAR ANNIVERSARY ON GTA 5,🎥: Access exclusive content at: https://www.si...,22,2021-11-02T19:00:15Z,PT27M42S,"['sidemen', 'moresidemen', 'miniminter', 'ksi'...",245273.0,6202423.0,4365,Sidemen year anniversary on gta,"(0, 22)\t0.636986903402816\n (0, 375)\t0.38..."
808,Sidemen,Pnpc0pySVTo,SIDEMEN OLD vs YOUNG FOR 24 HOURS CHALLENGE,SIDEMEN OLD vs YOUNG CHALLENGE HOW EXCITING\n🍗...,24,2022-10-30T19:31:52Z,PT42M25S,"['sidemen', 'sidemen sunday', '#sidemensunday']",351463.0,8020290.0,9357,Sidemen old vs young for hours challenge,"(0, 1091)\t0.5827298948003234\n (0, 423)\t0..."
801,Sidemen,u9-FIgI_V4s,"SIDEMEN $20,000 BIG FAT QUIZ OF THE YEAR",🎄 BUY CHRISTMAS DRILLINGS PLEASE: https://www....,24,2022-12-18T18:48:24Z,PT49M24S,"['sidemen', 'sidemen sunday', '#sidemensunday']",240637.0,5765737.0,4332,Sidemen big fat quiz of the year,"(0, 292)\t0.5046113473368071\n (0, 71)\t0.5..."
